In [ ]:
# 这行代码将你的Google Drive挂载到Colab虚拟机上
from google.colab import drive
drive.mount('/content/drive')

# 待办：在你的Drive中输入保存了解压后的作业文件夹的路径，
# 例如 'cs231n/assignments/assignment2/'
FOLDERNAME = 'cs231n/assignments/assignment2/'
assert FOLDERNAME is not None, "[!] 请输入文件夹名称。"

# 现在我们已经挂载了你的Drive，这行代码确保
# Colab虚拟机的Python解释器可以从其中加载
# Python文件
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# 如果CIFAR-10数据集尚未存在，这行代码会将其下载到你的Drive中
%cd /content/drive/My\ Drive/$FOLDERNAME/cs231n/datasets/
!bash get_datasets.sh  # 执行bash脚本下载数据集
%cd /content/drive/My\ Drive/$FOLDERNAME  # 切换回作业主目录

#  dropout（随机失活）

dropout[1]是一种通过在正向传播过程中随机将一些输出激活值设为零来正则化神经网络的技术。在本练习中，你需要实现一个dropout层，并修改你的全连接网络以选择性地使用dropout。

[1] [Geoffrey E. Hinton et al, "Improving neural networks by preventing co-adaptation of feature detectors", arXiv 2012](https://arxiv.org/abs/1207.0580)

In [ ]:
# 配置单元格
import time  # 导入时间模块，用于计时
import numpy as np  # 导入numpy库，用于数值计算
import matplotlib.pyplot as plt  # 导入matplotlib库，用于绘图
from cs231n.classifiers.fc_net import *  # 从cs231n的分类器模块导入全连接网络相关类/函数
from cs231n.data_utils import get_CIFAR10_data  # 从数据工具模块导入获取CIFAR-10数据的函数
from cs231n.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array  # 导入梯度检查相关函数
from cs231n.solver import Solver  # 导入求解器类，用于模型训练

# 设置matplotlib在 notebook 中内联显示
%matplotlib inline  
plt.rcParams["figure.figsize"] = (10.0, 8.0)  # 设置图表的默认大小
plt.rcParams["image.interpolation"] = "nearest"  # 设置图像插值方式为最近邻
plt.rcParams["image.cmap"] = "gray"  # 设置默认的图像颜色映射为灰度

%load_ext autoreload  # 加载自动重载扩展
%autoreload 2  # 设置自动重载模式，当导入的模块被修改时自动重新导入

def rel_error(x, y):
    """返回相对误差"""
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))  # 计算x和y之间的相对误差，避免除零

In [ ]:
# 加载（预处理后的）CIFAR-10数据
data = get_CIFAR10_data()
# 遍历数据字典中的所有键值对并打印
for k, v in list(data.items()):
    print(f"{k}：{v.shape}")  # 打印每个数据集的名称及其形状

# Dropout：前向传播

在文件`cs231n/layers.py`中，实现dropout的前向传播。由于dropout在训练模式和测试模式下的行为不同，要确保为两种模式都实现相应的操作。

完成实现后，运行下面的单元格来测试你的代码。

In [ ]:
np.random.seed(231)  # 设置随机种子，保证结果可复现
x = np.random.randn(500, 500) + 10  # 生成随机输入数据，均值约为10，形状为(500, 500)

# 测试不同的dropout概率p
for p in [0.25, 0.4, 0.7]:
    # 训练模式下的dropout前向传播（p为保留神经元的概率）
    out, _ = dropout_forward(x, {'mode': 'train', 'p': p})
    # 测试模式下的dropout前向传播
    out_test, _ = dropout_forward(x, {'mode': 'test', 'p': p})

    print('测试 p = ', p, ' 时的情况')
    print('输入数据的均值：', x.mean())
    print('训练模式下输出的均值：', out.mean())
    print('测试模式下输出的均值：', out_test.mean())
    print('训练模式下输出中被设为0的比例：', (out == 0).mean())
    print('测试模式下输出中被设为0的比例：', (out_test == 0).mean())
    print()  # 空行分隔不同p的测试结果

# Dropout：反向传播

在文件`cs231n/layers.py`中，实现dropout的反向传播。完成后，运行以下单元格，通过数值梯度检查来验证你的实现。

In [ ]:
np.random.seed(231)  # 设置随机种子，确保结果可复现
x = np.random.randn(10, 10) + 10  # 生成随机输入数据，均值约为10，形状为(10, 10)
dout = np.random.randn(*x.shape)  # 生成与x形状相同的随机输出梯度

# dropout参数：训练模式，保留概率p=0.2，随机种子123（保证dropout掩码固定）
dropout_param = {'mode': 'train', 'p': 0.2, 'seed': 123}
out, cache = dropout_forward(x, dropout_param)  # 执行dropout前向传播，获取输出和缓存
dx = dropout_backward(dout, cache)  # 执行dropout反向传播，计算输入梯度dx
# 用数值方法计算梯度作为参考（数值梯度）
dx_num = eval_numerical_gradient_array(lambda xx: dropout_forward(xx, dropout_param)[0], x, dout)

# 误差应在1e-10左右或更小
print('dx的相对误差：', rel_error(dx, dx_num))

## 内联问题1：
如果在dropout层中，不将通过反向dropout的数值除以`p`，会发生什么？为什么会出现这种情况？

## 答案：
[请在此处填写答案]


# 带Dropout的全连接网络

在文件`cs231n/classifiers/fc_net.py`中，修改你的实现以使用dropout。具体来说，如果网络的构造函数收到的`dropout_keep_ratio`参数值不是1，那么网络应该在每个ReLU非线性激活之后立即添加一个dropout层。完成后，运行以下代码通过数值梯度检查来验证你的实现。

In [ ]:
np.random.seed(231)  # 设置随机种子，确保结果可复现
# N：批量大小，D：输入维度，H1：第一层隐藏层大小，H2：第二层隐藏层大小，C：类别数
N, D, H1, H2, C = 2, 15, 20, 30, 10
X = np.random.randn(N, D)  # 生成随机输入数据
y = np.random.randint(C, size=(N,))  # 生成随机标签

# 测试不同的dropout保留比例
for dropout_keep_ratio in [1, 0.75, 0.5]:
    print('测试 dropout = ', dropout_keep_ratio, ' 时的情况')
    # 创建全连接网络模型
    model = FullyConnectedNet(
        [H1, H2],  # 隐藏层大小列表
        input_dim=D,  # 输入维度
        num_classes=C,  # 类别数
        weight_scale=5e-2,  # 权重初始化缩放因子
        dtype=np.float64,  # 数据类型
        dropout_keep_ratio=dropout_keep_ratio,  # dropout保留比例
        seed=123  # 随机种子，保证dropout行为可复现
    )

    loss, grads = model.loss(X, y)  # 计算损失和梯度
    print('初始损失：', loss)

    # 相对误差应在1e-6左右或更小
    # 注意：当dropout_keep_ratio=1时，W2的误差在1e-5量级左右是可接受的
    for name in sorted(grads):
        # 定义一个仅返回损失的函数，用于数值梯度计算
        f = lambda _: model.loss(X, y)[0]
        # 数值计算梯度
        grad_num = eval_numerical_gradient(f, model.params[name], verbose=False, h=1e-5)
        # 打印梯度的相对误差
        print('%s 的相对误差：%.2e' % (name, rel_error(grad_num, grads[name])))
    print()  # 空行分隔不同dropout保留比例的测试结果

# 正则化实验

作为一项实验，我们将在500个训练样本上训练两个双层网络：一个不使用dropout，另一个使用0.25的保留概率。然后我们将可视化这两个网络随时间变化的训练准确率和验证准确率。

In [ ]:
# 训练两个相同的网络，一个使用dropout，一个不使用
np.random.seed(231)  # 设置随机种子，确保实验可复现
num_train = 500  # 训练样本数量
small_data = {
    'X_train': data['X_train'][:num_train],  # 取前500个训练样本
    'y_train': data['y_train'][:num_train],
    'X_val': data['X_val'],  # 验证集数据
    'y_val': data['y_val'],
}

solvers = {}  # 用于存储不同dropout设置的求解器
dropout_choices = [1, 0.25]  # 测试两种dropout保留比例：1（无dropout）和0.25
for dropout_keep_ratio in dropout_choices:
    # 创建全连接网络模型，包含一个500维的隐藏层
    model = FullyConnectedNet(
        [500],
        dropout_keep_ratio=dropout_keep_ratio  # 设置dropout保留比例
    )
    print(dropout_keep_ratio)  # 打印当前的dropout保留比例

    # 创建求解器用于训练模型
    solver = Solver(
        model,  # 要训练的模型
        small_data,  # 使用的数据集
        num_epochs=25,  # 训练轮数
        batch_size=100,  # 批大小
        update_rule='adam',  # 使用adam优化算法
        optim_config={'learning_rate': 5e-4,},  # 优化器配置，学习率为5e-4
        verbose=True,  # 训练过程中打印详细信息
        print_every=100  # 每100次迭代打印一次信息
    )
    solver.train()  # 训练模型
    solvers[dropout_keep_ratio] = solver  # 将训练好的求解器存入字典
    print()  # 空行分隔不同dropout设置的训练过程

In [ ]:
# 绘制两个模型的训练准确率和验证准确率
train_accs = []  # 存储训练准确率
val_accs = []    # 存储验证准确率
for dropout_keep_ratio in dropout_choices:
    solver = solvers[dropout_keep_ratio]
    train_accs.append(solver.train_acc_history[-1])  # 取最后一轮的训练准确率
    val_accs.append(solver.val_acc_history[-1])      # 取最后一轮的验证准确率

plt.subplot(3, 1, 1)  # 创建3行1列的子图，当前绘制第1个
for dropout_keep_ratio in dropout_choices:
    # 绘制训练准确率曲线，标记为'o'，标签为对应的dropout保留比例
    plt.plot(
        solvers[dropout_keep_ratio].train_acc_history, 'o', label='%.2f dropout保留比例' % dropout_keep_ratio)
plt.title('训练准确率')
plt.xlabel('轮次')
plt.ylabel('准确率')
plt.legend(ncol=2, loc='lower right')  # 图例显示在右下角，2列

plt.subplot(3, 1, 2)  # 当前绘制第2个子图
for dropout_keep_ratio in dropout_choices:
    # 绘制验证准确率曲线，标记为'o'，标签为对应的dropout保留比例
    plt.plot(
        solvers[dropout_keep_ratio].val_acc_history, 'o', label='%.2f dropout保留比例' % dropout_keep_ratio)
plt.title('验证准确率')
plt.xlabel('轮次')
plt.ylabel('准确率')
plt.legend(ncol=2, loc='lower right')  # 图例显示在右下角，2列

plt.gcf().set_size_inches(15, 15)  # 设置图表大小
plt.show()  # 显示图表

## 内联问题2：
比较使用dropout和不使用dropout时的验证准确率与训练准确率——你的结果说明了dropout作为正则化器有什么作用？

## 答案：
[请在此处填写答案]
